In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

# setup wandb

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={
    'project': 'Object Detection',
    'entity': 'juyounghong',
    'config': {'lr': 1e-4, 'batch_size':16},
    'tags': ['resnet50', 'sgd'] 
},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100,
         bbox_score_thr=0.5)]

In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-11-20 14:07:18,652 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /opt/ml/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


  0%|          | 0.00/109M [00:00<?, ?B/s]

2022-11-20 14:07:32,973 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-11-20 14:07:32,996 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-11-20 14:07:33,005 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-11-20 14:07:33,114 - mmcv - INFO - 
backbone.patch_embed.projection.weight - torch.Size([96, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2022-11-20 14:07:33,115 - mmcv - INFO - 
backbone.patch_embed.projection.bias - torch.Size([96]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2022-11-20 14:07:33,115 - mmcv - INFO - 
backbone.patch_

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2022-11-20 14:07:36,152 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-11-20 14:07:36,607 - mmdet - INFO - Start running, host: root@36e87c75a3b2, work_dir: /opt/ml/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2022-11-20 14:07:36,608 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                  

2022-11-20 14:07:42,955 - mmdet - WARNING - No meta information found in the runner. 
2022-11-20 14:07:42,956 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2022-11-20 14:08:02,015 - mmdet - INFO - Epoch [1][50/1221]	lr: 1.978e-03, eta: 1:32:44, time: 0.381, data_time: 0.051, memory: 3749, loss_rpn_cls: 0.3552, loss_rpn_bbox: 0.0448, loss_cls: 0.5285, acc: 90.1250, loss_bbox: 0.1068, loss: 1.0354, grad_norm: 6.7482
2022-11-20 14:08:18,571 - mmdet - INFO - Epoch [1][100/1221]	lr: 3.976e-03, eta: 1:26:21, time: 0.331, data_time: 0.006, memory: 3749, loss_rpn_cls: 0.1954, loss_rpn_bbox: 0.0577, loss_cls: 0.3003, acc: 94.5098, loss_bbox: 0.1997, loss: 0.7529, grad_norm: 2.3957
2022-11-20 14:08:34,989 - mmdet - INFO - Epoch [1][150/1221]	lr: 5.974e-03, eta: 1:23:49, time: 0.328, data_time: 0.006, memory: 3750, loss_rpn_cls: 0.1667, loss_rpn_bbox: 0.0545, loss_c